In [98]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests
from selenium import webdriver

In [111]:
res = requests.post('https://kbl.or.kr/game/archive-team')
print(res.content)

b'<!doctype html><html lang="ko"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,minimum-scale=1,user-scalable=no"/><title>KBL \xea\xb3\xb5\xec\x8b\x9d \xed\x99\x88\xed\x8e\x98\xec\x9d\xb4\xec\xa7\x80</title><meta name="author" content="KBL"/><meta name="description" content="KBL \xea\xb3\xb5\xec\x8b\x9d \xed\x99\x88\xed\x8e\x98\xec\x9d\xb4\xec\xa7\x80"/><meta name="referrer" content="no-referrer"/><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta http-equiv="Expires" content="-1"/><meta http-equiv="Pragma" content="no-cache"><meta http-equiv="Cache-Control" content="no-cache, no-store, must-revalidate"/><link rel="icon" href="/imgs/favicon/kbl/favicon.svg"><link rel="apple-touch-icon" type="image/png" id="apple-icon-60x60" sizes="60x60" href="/imgs/favicon/kbl/apple-icon-60x60.png"><link rel="apple-touch-icon" type="image/png" id="apple-icon-87x87" sizes="87x87" href="/imgs/favicon/kbl/apple-icon-87x87.png"><link r

In [92]:
soup = BeautifulSoup(res.content, 'html.parser')

In [97]:
soup.find_all('div', class_='archive-team-table01-wrap')

[]

In [105]:
driver = webdriver.Chrome('chromedriver_win32/')
# driver.implicitly_wait(3) # 있으나 없으나
driver.get('https://kbl.or.kr/game/archive-team')

C:\Users\Kwon\AppData\Local\Temp\ipykernel_436728\4060881888.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver_win32/')


In [108]:
from selenium.webdriver.common.by import By

In [112]:
driver.find_elements(By.CLASS_NAME, 'con-box')

[<selenium.webdriver.remote.webelement.WebElement (session="cd3c949803d492622ec3a130bb34e157", element="6bb2d47d-8d65-472c-b19f-9ed0496bd676")>]

In [114]:
html = driver.page_source # page_source를 가져와야 하는 것이었다ㅠㅠ 감격스러움 진짜
soup = BeautifulSoup(html, 'html.parser')

In [117]:
nt = soup.select('div.archive-team-table01')

In [118]:
for i in nt:
    print(i.text.strip())

순위팀G1서울 SK232안양 KGC233울산 현대모비스224고양 캐롯225원주 DB226전주 KCC237대구 한국가스공사228창원 LG229수원 KT2210서울 삼성23합계224
PTS2P2PA2P%3P3PA3P%FGFGAFG%FTFTAFT%PPPPAPP%REBOUNDASTTOSTLBSPFOFFDEFTOT84.725.648.752.66.819.934.132.468.547.313.218.471.521.136.957.210.323.934.218.39.07.52.718.983.321.039.852.89.928.434.931.068.345.411.415.773.116.828.958.210.426.336.818.310.15.83.018.583.125.247.553.07.020.933.332.168.447.011.917.667.720.134.658.111.525.637.120.310.66.92.020.082.818.034.552.212.032.836.530.067.344.610.814.375.613.523.158.78.622.931.516.59.37.52.517.581.825.549.151.96.920.334.132.469.446.710.213.774.219.433.857.511.024.435.416.911.56.52.219.880.822.545.050.07.621.036.130.065.945.613.216.779.117.531.056.510.624.334.918.19.75.61.717.380.523.843.155.16.821.032.430.664.247.712.516.874.117.830.159.19.223.332.517.110.96.82.519.578.823.646.251.16.720.532.730.366.745.411.416.967.518.633.555.411.626.037.616.711.27.52.920.376.218.940.147.18.726.732.527.566.841.212.416.973.414.929.949.811.822.734.515.510.05.82.1